In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import hashlib
import zlib
import bz2
import bcrypt
import base64
from Crypto.Cipher import AES
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

In [ ]:
img = cv2.imread('IMG.jpg')
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
plt.subplot(121), plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)), plt.title('Original Image')
plt.xticks([]), plt.yticks([])

plt.subplot(122), plt.imshow(cv2.cvtColor(gray_img, cv2.COLOR_BGR2RGB)), plt.title('Image to Be Used')
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
block_size = 16
height, width = gray_img.shape
blocks = []
for i in range(0, height, block_size):
    for j in range(0, width, block_size):
        block = gray_img[i:i+block_size, j:j+block_size]
        blocks.append(block)
        
        
dct_blocks = []
for block in blocks:
    dct = cv2.dct(np.float32(block))
    dct_blocks.append(dct)

In [ ]:
watermark_data = "01101000 01100101 01101100 01101100 01101111 00100000 01100110 01110010 01101111 01101101 00100000 01110100 01101000 01100101 00100000 01101111 01110100 01101000 01100101 01110010 00100000 01110011 01101001 01100100 01100101 00100000 01101111 01100110 00100000 01110100 01101000 01100101 00100000 01101101 01101111 01101111 01101110 00100000 01100001 01101110 01100100 00100000 01110011 01110101 01101110 01110011"

# Compression and Encryption using zlib
watermark_data_compressed_zlib = zlib.compress(watermark_data.encode())
password = "cse8090ismylastproject"
key = hashlib.sha256(password.encode()).digest()
cipher = AES.new(key, AES.MODE_EAX)
nonce = cipher.nonce
ciphertext_zlib, tag = cipher.encrypt_and_digest(watermark_data_compressed_zlib)

# Compression and Encryption using bz2
watermark_data_compressed_bz2 = bz2.compress(watermark_data.encode())
cipher = AES.new(key, AES.MODE_EAX)
nonce = cipher.nonce
ciphertext_bz2, tag = cipher.encrypt_and_digest(watermark_data_compressed_bz2)


In [ ]:
binary_watermark_hashlib = [int(bit) for bit in bin(int.from_bytes(hashlib.md5(watermark_data_compressed_zlib).digest(), byteorder='big'))[2:]]

salt = bcrypt.gensalt()
watermark_data_compressed_bz2_encoded = base64.b64encode(watermark_data_compressed_bz2)
binary_watermark_bcrypt = [int(bit) for bit in bin(int.from_bytes(bcrypt.hashpw(watermark_data_compressed_bz2_encoded, salt), byteorder='big'))[2:]]

In [ ]:
binary_watermark = binary_watermark_bcrypt
#binary_watermark = binary_watermark_hashlib

In [ ]:
# batch processing and precomputed metrics (new optimization)
block_mses = []
for i, block in enumerate(dct_blocks):
    mse = np.mean((block - np.round(block)).flatten() ** 2)
    block_mses.append((i, mse))

sorted_block_positions = [i for i, _ in sorted(block_mses, key=lambda x: x[1])]

batch_size = 16
for i in range(0, len(sorted_block_positions), batch_size):
    batch_positions = sorted_block_positions[i:i+batch_size]

    best_block_index = batch_positions[0]
    dct_blocks[best_block_index][0][0] += 1.0 / 256.0

#Watermarking 
watermarked_dct = np.zeros_like(dct_blocks)
for i, dct in enumerate(dct_blocks):
    watermarked_dct[i] = dct
watermarked_blocks = [cv2.idct(np.float32(block)) for block in watermarked_dct]
watermarked_img = np.zeros_like(gray_img)
k = 0
for i in range(0, height, block_size):
    for j in range(0, width, block_size):
        watermarked_img[i:i+block_size, j:j+block_size] = watermarked_blocks[k]
        k += 1

for i, block in enumerate(dct_blocks):
    if i in block_mses:
        continue
    block[0][0] -= 1.0 / 256.0

#Inverse applied again, file saved to marked_img
marked_dct = np.zeros_like(dct_blocks)
for i, dct in enumerate(dct_blocks):
    marked_dct[i] = dct
marked_blocks = [cv2.idct(np.float32(block)) for block in marked_dct]
marked_img = np.zeros_like(gray_img)
k = 0
for i in range(0, height, block_size):
    for j in range(0, width, block_size):
        marked_img[i:i+block_size, j:j+block_size] = marked_blocks[k]
        k += 1
marked_img = cv2.normalize(marked_img, None, 0, 255, cv2.NORM_MINMAX)
marked_img = marked_img.astype(np.uint8)
    
cv2.imwrite("watermarked_image.jpg", watermarked_img)
cv2.imwrite("marked_image.jpg", marked_img)

In [ ]:
plt.imshow(cv2.cvtColor(watermarked_img, cv2.COLOR_BGR2RGB))
plt.xticks([]), plt.yticks([])
plt.title('Watermarked Image')
plt.show()

In [ ]:
ssim_score, _ = ssim(gray_img, watermarked_img, full=True, multichannel=True)
psnr_score = psnr(gray_img, watermarked_img, data_range=gray_img.max())

print("SSIM Score:", ssim_score)
print("PSNR Score:", psnr_score)

In [ ]:
watermark_length = 64

# Extract the watermark from the watermarked image
extracted_watermark = ""
for i, block in enumerate(blocks):
    if i in sorted_block_positions:
        extracted_bit = "0"  # Replace with the extracted bit (0 or 1)
        extracted_watermark += extracted_bit

# Decrypt the extracted watermark using AES
password = "cse8090ismylastproject"
key = hashlib.sha256(password.encode()).digest()
cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
extracted_watermark_encrypted = cipher.decrypt(ciphertext_bz2)

# Decompress the extracted watermark using bz2
extracted_watermark_compressed = bz2.decompress(extracted_watermark_encrypted)
extracted_watermark = extracted_watermark_compressed.decode()

# Compare the extracted watermark with the original watermark data
if extracted_watermark == watermark_data:
    print("Watermark successfully detected.")
else:
    print("Watermark not detected or does not match.")

In [ ]:
print(extracted_watermark)

In [ ]:
def calculate_r_test(gray_img, watermarked_img, binary_watermark):
    mean_original = np.mean(gray_img)
    mean_watermarked = np.mean(watermarked_img)

    binary_watermark = np.resize(binary_watermark, watermarked_img.shape)

    watermarked_image_zeroed = np.where(binary_watermark == 0, 0, watermarked_img)
    mean_watermarked_zeroed = np.mean(watermarked_image_zeroed)

    numerator = np.sum((watermarked_img - mean_watermarked) * (gray_img - mean_original))
    denominator = np.sum((watermarked_img - mean_watermarked) ** 2)

    # Calculate the R-test value
    r_test_value = (numerator / denominator) * (mean_watermarked_zeroed - mean_watermarked)

    return r_test_value


r_test_value = calculate_r_test(gray_img, marked_img, binary_watermark)
print("R-test value:", r_test_value)

In [ ]:
plt.imshow(cv2.cvtColor(gray_img, cv2.COLOR_BGR2RGB))
plt.xticks([]), plt.yticks([])
plt.title('Original Image')
plt.show()

In [ ]:
edited_img = cv2.imread('edited_img.jpg')

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(cv2.cvtColor(watermarked_img, cv2.COLOR_BGR2RGB))
plt.title('Watermarked Image')
plt.xticks([]), plt.yticks([])

# Display the edited image
plt.subplot(122)
plt.imshow(cv2.cvtColor(edited_img, cv2.COLOR_BGR2RGB))
plt.title('Edited Image')
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
# Resize both images to have the same dimensions
height, width = watermarked_img.shape[:2]
edited_img = cv2.resize(edited_img, (width, height))

In [ ]:
if watermarked_img.shape != edited_img.shape:
    # Resize the edited image to match the dimensions of the watermarked image
    edited_img = cv2.resize(edited_img, (watermarked_img.shape[1], watermarked_img.shape[0]))


In [ ]:
psnr_score = psnr(watermarked_img, edited_img, data_range=255)

print("PSNR Score:", psnr_score)